## Santatriniaina Avotra Randrianambinina


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [22]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [23]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [24]:
train_data, vocab = load_data("./data/train1.txt")
print("Load training set with length: %d" %len(train_data))
print(train_data[0])
print("\n")
print("liked: %d, decision: %d" %(vocab['liked'],vocab['decision']))
valid_data, _ = load_data("./data/valid1.txt")
print("Load validation set with length: %d" %len(valid_data))
print(valid_data[0])

Load training set with length: 5561
['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


liked: 7, decision: 2
Load validation set with length: 62
['<s>', 'we', 'agonized', 'over', 'the', 'decision', 'to', 'do', 'that.', '</s>']


In [25]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
## FILL CODE
    data_with_unk = []
    for sentences in data:   
        for (idx , word) in enumerate(sentences):
            #we do not need word in vocab since values of vocab account the word in sentence 
            #in the data as the function load_data():
            sentences[idx]=word if word in vocab and vocab[word] > mincount else "<unk>"
        data_with_unk.append(sentences)
        
    return data_with_unk

In [26]:
print("Data before removing the rare words")
# print(len(train_data))
print(train_data[0])
print(train_data[50])
print(train_data[-1])
print("="*115)
print("Data after removing the rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])
print(train_data[50])
print(train_data[-1])
print(" ")
# print(len(train_data))

Data before removing the rare words
['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']
['<s>', 'tom', 'showed', 'me', 'a', 'picture', 'of', 'the', 'food', 'they', 'had', 'eaten.', '</s>']
['<s>', 'i', "don't", 'remember', 'the', 'last', 'time', 'tom', 'and', 'i', 'did', 'that', 'together.', '</s>']
Data after removing the rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']
['<s>', 'tom', 'showed', 'me', 'a', 'picture', 'of', 'the', 'food', 'they', 'had', 'eaten.', '</s>']
['<s>', 'i', "don't", 'remember', 'the', 'last', 'time', 'tom', 'and', 'i', 'did', 'that', 'together.', '</s>']
 


In [27]:
def build_ngram(data, n):
    
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    
#     total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for i in range(len(sentence)):
#             total_number_words +=1
            for j in range(n):
                if i-j>=0:
                    context=sentence[i-j:i]
                    counts[context][sentence[i]] +=1
                else:
                    break
    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
        ## FILL CODE
        d=0
        for w in counts[context].keys():
            d += counts[context][w]
        for w in counts[context].keys():
            prob[context][w] = counts[context][w]/d
    return prob

In [28]:
# RUN TO BUILD NGRAM MODEL
n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
print(model[()]["liked"])
# for key, val in model.items():
#     print(key,val)
#     break

build ngram model with n =  4
0.00010608150090169275


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [29]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Returns:
    prob(float): probability of this word given the context 
    '''
    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model

    ## FILL CODE
    prob=model[context][w] if context in model and w in model[context] else 0.4*get_prob(model, context[1:], w)
    return prob

In [30]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    perp, T = 1.0, 0
    for sentence in data:
        sentence = tuple(sentence)
        for i in range(1, len(sentence)):
            nb_ctx=i if i<n-1 else n-1
            context = sentence[i-nb_ctx:i]
            perp += -np.log(get_prob(model, context, sentence[i]))
            T += 1
    perp = np.exp(perp/T)
    return perp

In [31]:
# COMPUTE PERPLEXITY ON VALIDATION SET
n=3
print("The perplexity of a train data is", perplexity(model, train_data, n))
print("The perplexity of a validation data is", perplexity(model, valid_data, n))

The perplexity of a train data is 5.1709828328983205
The perplexity of a validation data is 62.151547962838315


In [32]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if context not in model:
        words_and_probs=get_proba_distrib(model, context[1:])
    else:
        words_and_probs=model[context]
    return words_and_probs

In [33]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    N=len(sentence)
    while sentence[-1] != "</s>" and N<100:
        ## FILL CODE
        proba = get_proba_distrib(model, tuple(sentence))
        w = np.random.choice((list(proba.keys())), 1, p = list(proba.values()))
        sentence.append(w[0])
        
    return sentence

In [34]:
# GENERATE A SENTENCE FROM THE MODEL
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'mother', 'was', 'in', 'a', 'bad', 'mood', 'since', 'she', 'could', 'not', 'play', 'golf', 'because', 'of', 'bad', 'weather.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [35]:
#Set mincount = 1 and n = 2
mincount = 1
n = 3
train_data2, vocab2 = load_data("./data/train2.txt")
print("Load training set with length: %d" %len(train_data2))
print("Train data before removing the rare words")
print(train_data2[10])
print(train_data2[190])
print(train_data2[-3])
print("\n")
print("liked: %d, decision: %d" %(vocab['liked'],vocab['decision']))
print("="*105)
train_data2r = remove_rare_words(train_data2, vocab2, mincount)
print("Train data after removing the rare words")
print(train_data2r[10])
print(train_data2r[190])
print(train_data2r[-3])
print(" ")

valid_data2, _ = load_data("./data/valid2.txt")
print("Load validation set with length: %d" %len(valid_data2))
print("Validation data before removing the rare words")
print(valid_data2[0])
print(valid_data2[50])
print(valid_data2[-1])
print("="*105)
valid_data2 = remove_rare_words(valid_data2, vocab2, mincount)
print("Validation data after removing the rare words by setting by <unk>")
print(valid_data2[0])
print(valid_data2[50])
print(valid_data2[-1])
print(" ")

print("Build ngram model with n = ", n)
model2 = build_ngram(train_data2, n)

print("The perplexity of a validation data is given by", perplexity(model2, valid_data2, n))

print("Generated sentence: ",generate(model2))

Load training set with length: 180000
Train data before removing the rare words
['<s>', 'this', 'store', 'enjoys', 'a', '<unk>', 'advantage', '.', '</s>']
['<s>', 'do', 'you', 'have', 'a', 'subway', 'map', '?', '</s>']
['<s>', 'nick', 'looks', 'down', 'on', 'anyone', 'who', 'comes', 'from', 'a', 'rural', 'area', '.', '</s>']


liked: 7, decision: 2
Train data after removing the rare words
['<s>', 'this', 'store', 'enjoys', 'a', '<unk>', 'advantage', '.', '</s>']
['<s>', 'do', 'you', 'have', 'a', 'subway', 'map', '?', '</s>']
['<s>', 'nick', 'looks', 'down', 'on', 'anyone', 'who', 'comes', 'from', 'a', 'rural', 'area', '.', '</s>']
 
Load validation set with length: 20000
Validation data before removing the rare words
['<s>', 'the', 'wild', 'horse', 'was', 'finally', 'broken', 'by', 'the', 'patient', '<unk>', '.', '</s>']
['<s>', 'i', 'have', 'to', 'do', 'my', 'homework', '.', '</s>']
['<s>', 'nothing', 'is', 'more', '<unk>', 'than', 'to', 'lose', 'in', 'the', '<unk>', '.', '</s>']
Vali